In [ ]:
import torch
from matplotlib import pyplot as plt
import numpy as np
import cv2

In [ ]:
import torch
print(torch.__version__)
print(torch.cuda.is_available())
DEVICE = torch.device('cuda:0')

In [ ]:
import mmcv
print(mmcv.__version__)
# Check MMRotate installation
import mmrotate
print(mmrotate.__version__)

# Check MMDetection installation
import mmdet
print(mmdet.__version__)

# Check mmcv installation
from mmcv.ops import get_compiling_cuda_version, get_compiler_version
print(get_compiling_cuda_version())
print(get_compiler_version())

In [ ]:
from mmrotate.registry import DATASETS
from mmrotate.datasets.dota import DOTADataset


@DATASETS.register_module()
class VBdataset(DOTADataset):
    """Vertabrea for detection."""
    METAINFO = {
        'classes':
        ('VB',),
        'palette': [(165, 42, 42)]
    }

In [ ]:
from mmengine import Config
cfg = Config.fromfile('mmrotate/configs/redet/redet-le90_re50_refpn_1x_dota.py')

In [ ]:
print(f'Config:\n{cfg.pretty_text}')

In [ ]:
# Since we use only one GPU, BN is used instead of SyncBN
# cfg.norm_cfg = dict(type='BN', requires_grad=True)



# Modify dataset type and path
cfg.dataset_type = 'VBdataset'
cfg.data_root = 'train_data/train_data_12_797_ROI/'

cfg.train_dataloader.dataset.type = 'VBdataset'
cfg.train_dataloader.dataset.data_root ='train_data/train_data_12_797_ROI/'
cfg.train_dataloader.dataset.ann_file = 'train/'
cfg.train_dataloader.dataset.data_prefix.img_path = 'images/'
cfg.train_dataloader.num_workers=0
cfg.train_dataloader.persistent_workers=False

cfg.val_dataloader.dataset.type = 'VBdataset'
cfg.val_dataloader.dataset.data_root = 'train_data/train_data_12_797_ROI/'
cfg.val_dataloader.dataset.ann_file = 'val/'
cfg.val_dataloader.dataset.data_prefix.img_path = 'images/'
cfg.val_dataloader.num_workers=0
cfg.val_dataloader.persistent_workers=False

cfg.test_dataloader = cfg.val_dataloader

# modify num classes of the model in box head
cfg.model.roi_head.bbox_head[0].num_classes = 1
cfg.model.roi_head.bbox_head[1].num_classes = 1

cfg.model.backbone.init_cfg.checkpoint='re_resnet50_c8_batch256-25b16846.pth'
# # cfg.load_from = 'download_models/redet_re50_fpn_1x_dota_ms_rr_le90-fc9217b5.pth'

# cfg.load_from = 'redet_re50_fpn_1x_dota_ms_rr_le90-fc9217b5.pth'
cfg.load_from =  None
# cfg.resume_from ='tutorial_exps11/latest.pth'
# Set up working dir to save files and logs.
cfg.work_dir = './run/train/redet_scol'


cfg.optim_wrapper.optimizer.lr = 0.005
cfg.optim_wrapper.optimizer.momentum=0.99
cfg.param_scheduler[1].gamma = 0.5
cfg.param_scheduler[1].end = 50
cfg.param_scheduler[1].milestones = [5, 20, 40, 50]

cfg.train_cfg.max_epochs = 60
cfg.default_hooks.logger.interval = 10

# Set seed to facilitate reproducing the result
cfg['randomness'] = dict(seed=0)

print(f'Config:\n{cfg.pretty_text}')

In [ ]:
from mmengine.runner import Runner

runner = Runner.from_cfg(cfg)

In [ ]:
# start training
runner.train()

In [ ]:
runner.test()